In [1]:
import keras
from keras.preprocessing import image
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

%env TF_GPU_ALLOCATOR=cuda_malloc_async

2025-07-11 09:37:45.269189: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-11 09:37:45.297536: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-11 09:37:45.309492: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-11 09:37:45.418385: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


env: TF_GPU_ALLOCATOR=cuda_malloc_async


In [2]:
WIDTH = 140
HEIGHT = 200
def read_image(filename: str):
    im = image.load_img(filename, target_size=(HEIGHT, WIDTH))
    # gray = im.convert('L')
    im_array = image.img_to_array(im) / 255
    return im_array
    # gray_array = image.img_to_array(gray) / 255 * 0.8
    # color = im_array - gray_array
    # return (color - color.min()) / (color.max() - color.min())

In [3]:
# 1-60 +
# 61-100 -
# 101-140 +
# 141-180 -
# 181-240 +
# 241-259 -
# 260-300 +
# 301-320 -
# 321-390 +
# 391-500 -

CORRECT = [[1]]
WRONG = [[0]]

labels = np.array(CORRECT*40*4 +
                  WRONG*60*4 +
                  CORRECT*40*4 +
                  WRONG*40*4 +
                  CORRECT*60*4 +
                  WRONG*19*4 +
                  CORRECT*41*4 +
                  WRONG*20*4 +
                  CORRECT*70*4 +
                  WRONG*110*4
)

In [4]:
images = []
DATA_PATH = '../data'
import os
for filename in sorted(os.listdir(DATA_PATH)):
    if (not filename.endswith('.png')) or filename.startswith('.'):
        continue

    images.append(read_image(f'{DATA_PATH}/{filename}'))

In [5]:
input = keras.Input((HEIGHT,WIDTH,3))

c1 = keras.layers.DepthwiseConv2D(3, depth_multiplier=6, activation='relu')(input)
c2 = keras.layers.DepthwiseConv2D((5,3), depth_multiplier=5, activation='gelu')(c1)
c2 = keras.layers.BatchNormalization()(c2)
p1 = keras.layers.MaxPool2D((2,2))(c2)

c3 = keras.layers.DepthwiseConv2D((5,3), depth_multiplier=4, activation='gelu')(p1)
c3 = keras.layers.BatchNormalization()(c3)
cp1 = keras.layers.Conv2D(128, (1,1), activation='linear')(c3)
p2 = keras.layers.MaxPool2D((4,4))(cp1)

f = keras.layers.Flatten()(p2)
d = keras.layers.Dropout(0.8)(f)
l = fc10 = keras.layers.Dense(1536, activation='relu')(d)
l = fc20 = keras.layers.Dense(256, activation='relu')(l)
# l = keras.layers.Dense(256, activation='relu')(l)
# l = keras.layers.Dense(256, activation='relu')(l)
l = fc30 = keras.layers.Dense(128, activation='relu')(l)
l = out = keras.layers.Dense(1, activation='sigmoid')(l)
model = keras.Model(input, l)

loss = keras.losses.BinaryFocalCrossentropy()

model.compile(loss=loss, optimizer=keras.optimizers.Lion(learning_rate=1e-3), metrics=['mae', 'mse', 'accuracy'])
model.summary()

I0000 00:00:1752208683.454813    8787 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1752208683.788458    8787 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1752208683.789964    8787 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1752208683.792891    8787 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 200, 140, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d                │ (None, 198, 138, 18)   │           180 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_1              │ (None, 194, 136, 90)   │         1,440 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 194, 136, 90)   │           360 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 97, 68, 90)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_2              │ (None, 93, 66, 360)    │         5,760 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 93, 66, 360)    │         1,440 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 93, 66, 128)    │        46,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 23, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 47104)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 47104)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1536)           │    72,353,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       393,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 72,835,165 (277.84 MB)

 Trainable params: 72,834,265 (277.84 MB)

 Non-trainable params: 900 (3.52 KB)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2)

In [7]:
def scheduler(epoch, lr):
    if epoch > 0:
        return lr * np.exp(-0.01 * epoch)
    
    return lr



learning_rate_callback = keras.callbacks.LearningRateScheduler(scheduler)
lion_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)

In [ ]:
history1 = model.fit(
    np.array(X_train),
    y_train,
    epochs=10,
    batch_size=16,
    callbacks=[
        learning_rate_callback,
        lion_early_stopping
    ])

model.compile(loss=loss, optimizer=keras.optimizers.Adam(learning_rate=1e-4, metrics=['mae', 'mse', 'accuracy']))

history2 = model.fit(
    np.array(X_train),
    y_train,
    epochs=100,
    batch_size=16,
    callbacks=[
        learning_rate_callback,
    ])

Epoch 1/10


I0000 00:00:1752208688.588951    9616 service.cc:146] XLA service 0x7f7834011880 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752208688.588986    9616 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-07-11 09:38:08.649575: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-07-11 09:38:08.904573: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90101
E0000 00:00:1752208699.843428    9616 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752208700.032193    9616 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsig

100/100 ━━━━━━━━━━━━━━━━━━━━ 32s 101ms/step - accuracy: 0.5009 - loss: 4.0072 - mae: 0.4971 - mse: 0.4143 - learning_rate: 0.0010
Epoch 2/10


/home/nikita/.conda/envs/ai_new/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,mae,mse,learning_rate
  current = self.get_monitor_value(logs)


100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 100ms/step - accuracy: 0.5003 - loss: 2.7034 - mae: 0.4992 - mse: 0.2607 - learning_rate: 9.9005e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 100ms/step - accuracy: 0.4990 - loss: 0.2969 - mae: 0.5000 - mse: 0.2505 - learning_rate: 9.7045e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 100ms/step - accuracy: 0.5058 - loss: 0.3678 - mae: 0.5000 - mse: 0.2501 - learning_rate: 9.4176e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 100ms/step - accuracy: 0.5088 - loss: 1.0919 - mae: 0.4992 - mse: 0.2501 - learning_rate: 9.0484e-04
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 101ms/step - accuracy: 0.4717 - loss: 0.1734 - mae: 0.5001 - mse: 0.2501 - learning_rate: 8.6071e-04
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 100ms/step - accuracy: 0.5043 - loss: 5.1389 - mae: 0.5003 - mse: 0.2505 - learning_rate: 8.1058e-04
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 101ms/step - accuracy: 0.4969 - loss: 1.4667 - mae: 0.4997 - mse: 0.2501 - learning_rate: 7.5578e

ValueError: Argument(s) not recognized: {'metrics': ['mae', 'mse', 'accuracy']}

In [ ]:
# === TRAIN DATA ===

test = model.predict(np.array(X_train))
error = abs(test - y_train)

total_size = len(X_train)
correct = 0
for sample in error:
    if sample[0] < 0.15:
        correct += 1

print(f'TRAIN DATA: {correct / total_size * 100:.2f}% ({(correct / total_size * 100 - 50) * 2:.2f}%)')

In [ ]:
test = model.predict(np.array(X_test))
error = abs(test - y_test)

total_size = len(X_test)
correct = 0
for sample in error:
    if sample[0] < 0.15:
        correct += 1

print(f'Accuracy: {correct / total_size * 100:.2f}% ({(correct / total_size * 100 - 50) * 2:.2f}%)')
plt.plot(history1.history['loss'], label='Loss', color='blue')
plt.plot(history2.history['loss'], label='Loss', color='blue')
plt.plot(history1.history['accuracy'], label='Accuracy', color='orange')
plt.plot(history2.history['accuracy'], label='Accuracy', color='orange')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()


In [ ]:
error

In [ ]:
test

In [ ]:
size = 128
subm = keras.Model(input, p2)
subm.compile(loss='mae', optimizer='adam', metrics=['mae', 'mse'])

sample = 230

test_layer = subm.predict(np.array([X_test[sample]]))[0]
print(test_layer.shape)
x, y, _ = test_layer.shape

plt.title(f'Sample {sample} ({"Correct" if labels[sample] == CORRECT else "Wrong"})')
plt.imshow(X_test[sample])
plt.show()

In [ ]:
_, rows = plt.subplots(size // 8 + (1 if size % 8 else 0), 8, figsize=(20, 3 * (size // 8) + (1 if size % 8 else 0)))
for j,row in enumerate(rows.reshape(-1, 8)):
    for i, axis in enumerate(row):
        try:
            # frame = test_layer[:, :, (i*3 + j*24):((i+1)*3 + j*24)]
            frame = test_layer[:, :, (i + j*8)]
            axis.imshow(frame)
        except IndexError:
            break

In [ ]:
first_fc = keras.Model(p2, fc1)
first_fc.compile(loss='mae', optimizer='adam', metrics=['mae'])

first_fc_test = first_fc.predict(np.array([test_layer]))
plt.figure(figsize=(20, 10))
plt.imshow(first_fc_test.reshape(64, -1))
# print(first_fc_test.shape)

In [ ]:
wrong_idx = [i for i, err in enumerate(error) if err > 0.15]
wrong_images = [X_test[i] for i in wrong_idx]
wrong_labels = [y_test[i] for i in wrong_idx]
wrong_predictions = [test[i] for i in wrong_idx]

ii = iter(wrong_images)
li = iter(wrong_labels)
pi = iter(wrong_predictions)

_, rows = plt.subplots(12,16, figsize=(30, 40))
for row in rows:
    for axis in row:
        try:
            frame = next(ii)
            label = next(li)
            pred = next(pi)
            axis.imshow(frame)
            axis.set_title(f'{'OK' if label==CORRECT else 'ALARM'} || {pred}')
        except StopIteration:
            break